In [7]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras import optimizers
import shutil
import random
from PIL import Image
import numpy as np
import logging
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil
from tensorflow import debugging
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint

from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from keras.regularizers import l2,L1L2 

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)



2.8.0


# Step 1: create lumo predictor

In [8]:

X = np.array(pd.read_csv("pretrain_features.csv.zip").iloc[:, 2:]).astype(np.float32)
Y = np.array(pd.read_csv("pretrain_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

lumo_predictor = tf.keras.Sequential([
            layers.Dense(512, activation=tf.keras.activations.relu, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            layers.Dense(512, activation=tf.keras.activations.relu, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            # layers.Dense(512, activation=tf.keras.activations.relu),
            # layers.Dense(512, activation=tf.keras.activations.relu),
            layers.Dense(1, activation='linear')
        ])
        
lumo_predictor.compile(loss=losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(0.0001))

# retrain = True
retrain = False
if(retrain):
    lumo_predictor.fit(X, Y, epochs=100,verbose=False)
    # lumo_predictor.fit(X, Y, epochs=100)
    lumo_predictor.save_weights('lumo_predictor.h5')
else:
    #build the model
    lumo_predictor.predict(X)
    lumo_predictor.load_weights("lumo_predictor.h5")

# Step 2: create encoder

In [9]:
encoder = tf.keras.Sequential([
            #encoder
            layers.Dense(64, activation=tf.keras.activations.linear, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            #classifier
            layers.Dense(512, activation=tf.keras.activations.relu, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            # layers.Dense(1, activation='linear', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))
            layers.Dense(1, activation='linear')
        ])
        
encoder.compile(loss=losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(0.0001))

# retrain = True
retrain = False
if(retrain):
    encoder.fit(X, Y, epochs=30)
    encoder.save_weights('encoder.h5')
else:
    #build the model
    encoder.predict(X)
    encoder.load_weights("encoder.h5")

Epoch 1/100
500/500 [==============================] - 3s 5ms/step - loss: 1.7857
Epoch 2/100
500/500 [==============================] - 3s 5ms/step - loss: 0.2938
Epoch 3/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2581
Epoch 4/100
500/500 [==============================] - 3s 7ms/step - loss: 0.2327
Epoch 5/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2104
Epoch 6/100
500/500 [==============================] - 3s 6ms/step - loss: 0.1899
Epoch 7/100
500/500 [==============================] - 3s 6ms/step - loss: 0.1708
Epoch 8/100
500/500 [==============================] - 3s 6ms/step - loss: 0.1531
Epoch 9/100
500/500 [==============================] - 3s 6ms/step - loss: 0.1370
Epoch 10/100
500/500 [==============================] - 3s 6ms/step - loss: 0.1223
Epoch 11/100
500/500 [==============================] - 3s 6ms/step - loss: 0.1092
Epoch 12/100
500/500 [==============================] - 3s 6ms/step - loss: 0.0974
Epoch 13/100


# Step 3: setup data to train homo predictor

In [10]:
X_train = np.array(pd.read_csv("train_features.csv.zip").iloc[:, 2:]).astype(np.float32)
Y_train = np.array(pd.read_csv("train_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

Y_gap = np.array(pd.read_csv("train_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

Y_lumo = lumo_predictor.predict(X_train)

Y_homo = -(Y_gap - Y_lumo)



# Step 4: train homo predictor

In [11]:
homo_predictor = tf.keras.Sequential([
            #encoder
            layers.Dense(64, activation=tf.keras.activations.linear, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001),trainable=False),
            #classifier
            layers.Dense(512, activation=tf.keras.activations.relu, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            # layers.Dense(1, activation='linear', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))
            layers.Dense(1, activation='linear')
        ])

#build the model
homo_predictor.predict(X_train)

#transfer learning
homo_predictor.load_weights("encoder.h5")

homo_predictor.compile(loss=losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(0.00001))
history = homo_predictor.fit(X_train, Y_homo, epochs=20000,batch_size=100,verbose=False)



# Step 5: test set

In [12]:
X_test = np.array(pd.read_csv("test_features.csv.zip").iloc[:, 2:]).astype(np.float32)
sample = pd.read_csv("sample.csv")


homo = homo_predictor.predict(X_test)
lumo = lumo_predictor.predict(X_test)

sample["y"] = (lumo-homo)

sample.to_csv('submit.csv', index=False, float_format='%.5f')